<a href="https://colab.research.google.com/github/AslanDevbrat/Occupancy_Detection/blob/main/somewhere1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 %tensorflow_version 2.x
 from __future__ import absolute_import, division, print_function, unicode_literals
import functools
import os
import pandas as pd
import matplotlib.pyplot as plt

import numpy as np
import tensorflow as tf

print("TensorFlow version: {}".format(tf.__version__))
print("Eager execution: {}".format(tf.executing_eagerly()))

TensorFlow version: 2.2.0-rc1
Eager execution: True


In [ ]:
!pip install scikit-learn

In [ ]:
import pandas as pd

train_data = pd.read_csv('datatraining.csv')
train_data
train_data=train_data.drop(['index','date','HumidityRatio'],axis=1)
train_tem_mean,train_hum_mean,train_light_mean,train_co_mean,x = train_data.mean()
arr1 = [train_data.mean()]
train_tem_std,train_hum_std,train_light_std,train_co_std,x = train_data.std()
train_tem_std
arr2 =[train_data.std()]
arr2

#train_data


[Temperature     20.827630
 Humidity        27.998747
 Light          121.453183
 CO2            686.479722
 Occupancy        0.211120
 dtype: float64]

In [ ]:
from sklearn.preprocessing import StandardScaler
scaled_features = train_data.copy()
col_names = ['Temperature','Humidity','Light','CO2']
features = scaled_features[col_names]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)
scaled_features [col_names] = features
print(scaled_features)

       Temperature  Humidity     Light       CO2  Occupancy
0         0.899461  0.606332  1.562204  1.094134          1
1         0.928402  0.580541  1.562204  0.999552          1
2         0.906696  0.604237  1.545719  1.011508          1
3         0.906696  0.604237  1.570447  1.029839          1
4         0.928402  0.606332  1.562204  1.017618          1
...            ...       ...       ...       ...        ...
17890     0.214521  1.566565  1.540773  0.321271          1
17891     0.214521  1.546738  1.540773  0.328445          1
17892     0.262756  1.566082  1.540773  0.357138          1
17893     0.262756  1.597998  1.540773  0.426746          1
17894     0.262756  1.586392  1.610011  0.428872          1

[17895 rows x 5 columns]


In [ ]:
type(scaled_features)

pandas.core.frame.DataFrame

In [ ]:
feature_names = scaled_features.columns[:-1]
label_name = scaled_features.columns[4]
print("Features: {}".format(feature_names))
print("Label: {}".format(label_name))

Features: Index(['Temperature', 'Humidity', 'Light', 'CO2'], dtype='object')
Label: Occupancy


In [ ]:
label = scaled_features.pop('Occupancy')

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((scaled_features.values, label.values))
val_dataset = tf.data.Dataset.from_tensor_slices((scaled_features.values, label.values))
val_dataset = val_dataset.batch(64)

In [ ]:
for feat, targ in dataset.take(50):
  print ('Features: {}, Target: {}'.format(feat, targ))

Features: [0.89946108 0.60633211 1.56220397 1.09413392], Target: 1
Features: [0.92840221 0.58054106 1.56220397 0.99955193], Target: 1
Features: [0.90669636 0.60423659 1.54571877 1.01150752], Target: 1
Features: [0.90669636 0.60423659 1.57044657 1.02983942], Target: 1
Features: [0.92840221 0.60633211 1.56220397 1.01761815], Target: 1
Features: [0.89946108 0.63083361 1.56220397 1.04524885], Target: 1
Features: [0.92840221 0.61874405 1.54571877 1.05853284], Target: 1
Features: [0.92840221 0.65662466 1.56220397 1.0590642 ], Target: 1
Features: [0.92840221 0.641956   1.54571877 1.06969139], Target: 1
Features: [0.92840221 0.67016496 1.56220397 1.08669489], Target: 1
Features: [0.92840221 0.68209332 1.56426462 1.15178643], Target: 1
Features: [0.92840221 0.69176496 1.58404686 1.15577163], Target: 1
Features: [0.92840221 0.65694705 1.58528324 1.1956236 ], Target: 1
Features: [0.92840221 0.65646346 1.58404686 1.15577163], Target: 1
Features: [0.92840221 0.6772575  1.58281047 1.15258347], Targe

In [ ]:
train_dataset = dataset.shuffle(len(scaled_features)).batch(1)

In [ ]:
def get_compiled_model():
  model = tf.keras.Sequential([
  tf.keras.layers.Dense(4, activation=tf.nn.sigmoid, input_shape=(4,)),  # input shape required
  tf.keras.layers.Dense(5, activation=tf.nn.sigmoid),
  tf.keras.layers.Dense(2,activation=tf.nn.log_softmax)

  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [ ]:
model = get_compiled_model()

model.fit(train_dataset, epochs=3,validation_data=val_dataset)

Epoch 1/3
17895/17895 [==============================] - 46s 3ms/step - loss: 0.1156 - accuracy: 0.9559 - val_loss: 0.0404 - val_accuracy: 0.9902
Epoch 2/3
17895/17895 [==============================] - 46s 3ms/step - loss: 0.0403 - accuracy: 0.9905 - val_loss: 0.0400 - val_accuracy: 0.9904
Epoch 3/3
17895/17895 [==============================] - 47s 3ms/step - loss: 0.0404 - accuracy: 0.9904 - val_loss: 0.0399 - val_accuracy: 0.9903


In [ ]:
test_data = pd.read_csv('datatest.csv')
test_data=test_data.drop(['index','date','HumidityRatio'],axis=1)
#test_data


In [ ]:
scaled_features1 = test_data.copy()
col_names = ['Temperature','Humidity','Light','CO2']
features1 = scaled_features1[col_names]
scaler1 = StandardScaler().fit(features1)
features1 = scaler.transform(features1.values)
scaled_features1 [col_names] = features1
print(scaled_features1)

      Temperature  Humidity     Light       CO2  Occupancy
0        2.770987 -0.334010  2.293487  0.199962          1
1        2.788352 -0.330528  2.259858  0.235669          1
2        2.799929 -0.342134  2.231503  0.265213          1
3        2.792693 -0.362444  1.841216  0.281420          1
4        2.823081 -0.347937  1.815746  0.294969          1
...           ...       ...       ...       ...        ...
2660     3.340163 -0.444653  3.395358  1.478573          1
2661     3.378751 -0.437690  3.404260  1.411462          1
2662     3.378751 -0.444653  3.439868  1.400622          1
2663     3.404477 -0.444653  3.420086  1.391695          1
2664     3.454320 -0.448199  3.345902  1.394883          1

[2665 rows x 5 columns]


In [ ]:
label1 = scaled_features1.pop('Occupancy')

In [ ]:
test_dataset = tf.data.Dataset.from_tensor_slices((scaled_features1.values, label1.values))

In [ ]:
for feat, targ in test_dataset.take(50):
  print ('Features: {}, Target: {}'.format(feat, targ))

Features: [ 2.77098743 -0.33400958  2.29348732  0.19996211], Target: 1
Features: [ 2.78835211 -0.33052779  2.25985751  0.23566947], Target: 1
Features: [ 2.79992856 -0.34213376  2.23150297  0.26521306], Target: 1
Features: [ 2.79269328 -0.36244421  1.84121593  0.28141952], Target: 1
Features: [ 2.82308146 -0.34793675  1.8157463   0.29496919], Target: 1
Features: [ 2.82886969 -0.33633078  2.21172074  0.33003892], Target: 1
Features: [ 2.79992856 -0.33052779  2.05181433  0.35554418], Target: 1
Features: [ 2.82308146 -0.33052779  1.91663571  0.35235602], Target: 1
Features: [ 2.82308146 -0.31892182  1.75343226  0.37212259], Target: 1
Features: [ 2.80571679 -0.3111845   1.92158127  0.39061391], Target: 1
Features: [ 2.81439913 -0.30054569  1.78063283  0.41053989], Target: 1
Features: [ 2.77098743 -0.27830091  1.7821165   0.43843626], Target: 1
Features: [ 2.77098743 -0.2705636   1.74972309  0.46394152], Target: 1
Features: [ 2.77098743 -0.25122031  1.71881334  0.50645028], Target: 1
Featur

In [ ]:

test_dataset = test_dataset.shuffle(len(scaled_features1)).batch(1)



In [ ]:
retult = model.evaluate(features1,label1,batch_size=128)

21/21 [==============================] - 0s 2ms/step - loss: 0.0798 - accuracy: 0.9782


In [ ]:
class_names = ['0','1','2']
predict_dataset = tf.convert_to_tensor([
    [1.02487264, 0.01769,  0.6, 0.0983848,],
    [23.18, 27.272 ,426 ,721.25,],
    [1.02487264, 0.60472017, 1.53953682, 0.92781839]
])

# training=False is needed only if there are layers with different
# behavior during training versus inference (e.g. Dropout).
predictions = model(predict_dataset, training=False)

for i, logits in enumerate(predictions):
  class_idx = tf.argmax(logits).numpy()
  print(logits)
  p = tf.nn.softmax(logits)[class_idx]
  name = class_names[class_idx]
  print("Example {} prediction: {} ({:4.1f}%)".format(i, name, 100*p))

tf.Tensor([-0.00786541 -4.8492117 ], shape=(2,), dtype=float32)
Example 0 prediction: 0 (99.2%)
tf.Tensor([-3.8189964  -0.02219433], shape=(2,), dtype=float32)
Example 1 prediction: 1 (97.8%)
tf.Tensor([-3.5236626  -0.02993485], shape=(2,), dtype=float32)
Example 2 prediction: 1 (97.1%)


In [ ]:
export_path_sm = "./{}".format('occupanyDetection')
print(export_path_sm)

tf.saved_model.save(model, export_path_sm)

./occupanyDetection
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./occupanyDetection/assets


In [ ]:


export_path_keras = "./{}.h5".format('occupancyDetection')
print(export_path_keras)

model.save(export_path_keras)

./occupancyDetection.h5


In [ ]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model('/content/occupanyDetection')
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)


2188

In [ ]:
scaler.mean_

array([ 20.82763028,  27.9987467 , 121.45318338, 686.47972152])

In [ ]:
scaler.var_

array([1.07451273e+00, 2.67263356e+01, 4.08854891e+04, 9.83831106e+04])

In [ ]:
import math as m


In [ ]:
np.sqrt(scaler.var_)

array([  1.03658706,   5.16975199, 202.20160509, 313.66082097])